In [55]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import pickle

In [56]:
# Create a df with holidays
from workalendar.europe import Belgium
cal = Belgium()
years = list(range(2014, 2021))
holidays = []
for year in years:
    holidays.extend(cal.holidays(year))
holidays

[(datetime.date(2014, 1, 1), 'New year'),
 (datetime.date(2014, 4, 21), 'Easter Monday'),
 (datetime.date(2014, 5, 1), 'Labour Day'),
 (datetime.date(2014, 5, 29), 'Ascension Thursday'),
 (datetime.date(2014, 6, 9), 'Whit Monday'),
 (datetime.date(2014, 7, 21), 'National Day'),
 (datetime.date(2014, 8, 15), 'Assumption of Mary to Heaven'),
 (datetime.date(2014, 11, 1), 'All Saints Day'),
 (datetime.date(2014, 11, 11), 'Armistice of 1918'),
 (datetime.date(2014, 12, 25), 'Christmas Day'),
 (datetime.date(2015, 1, 1), 'New year'),
 (datetime.date(2015, 4, 6), 'Easter Monday'),
 (datetime.date(2015, 5, 1), 'Labour Day'),
 (datetime.date(2015, 5, 14), 'Ascension Thursday'),
 (datetime.date(2015, 5, 25), 'Whit Monday'),
 (datetime.date(2015, 7, 21), 'National Day'),
 (datetime.date(2015, 8, 15), 'Assumption of Mary to Heaven'),
 (datetime.date(2015, 11, 1), 'All Saints Day'),
 (datetime.date(2015, 11, 11), 'Armistice of 1918'),
 (datetime.date(2015, 12, 25), 'Christmas Day'),
 (datetime.dat

In [57]:
df_holidays = pd.DataFrame(holidays, columns=['Datetime', 'holiday'])
df_holidays.head()
df_holidays.to_pickle('./holidays2014_2020.pkl')

In [58]:
df_holidays['Datetime'] = pd.to_datetime(df_holidays['Datetime'])
df_holidays['date'] = df_holidays['Datetime'].dt.date
df_holidays

,Datetime,holiday,date
0,2014-01-01,New year,2014-01-01
1,2014-04-21,Easter Monday,2014-04-21
2,2014-05-01,Labour Day,2014-05-01
3,2014-05-29,Ascension Thursday,2014-05-29
4,2014-06-09,Whit Monday,2014-06-09
...,...,...,...
65,2020-07-21,National Day,2020-07-21
66,2020-08-15,Assumption of Mary to Heaven,2020-08-15
67,2020-11-01,All Saints Day,2020-11-01
68,2020-11-11,Armistice of 1918,2020-11-11


In [65]:
data1 = pd.read_pickle('Building-1-Roulart Media Group/building_1_cnsumptions.pkl')
data1.index = pd.to_datetime(data1.index)
data1['Datetime'] = data1.index
data1

,Valeur,Datetime
Date,,
2014-01-06 02:00:00,6.51,2014-01-06 02:00:00
2014-01-06 02:15:00,6.51,2014-01-06 02:15:00
2014-01-06 02:30:00,6.72,2014-01-06 02:30:00
2014-01-06 02:45:00,6.51,2014-01-06 02:45:00
2014-01-06 03:00:00,6.93,2014-01-06 03:00:00
...,...,...
2020-06-30 23:00:00,5.04,2020-06-30 23:00:00
2020-06-30 23:15:00,3.36,2020-06-30 23:15:00
2020-06-30 23:30:00,3.36,2020-06-30 23:30:00


In [70]:
# Making sure there are no duplicated data
# If there are some duplicates we average the data during those duplicated days
data1 = data1.groupby('Datetime', as_index=False)['Valeur'].mean()

# Sorting the values
data1['Datetime'] = pd.to_datetime(data1['Datetime'])
data1 = data1.loc[data1['Datetime'] <= '2020-07-01']
data1 = data1.sort_values('Datetime')
data1['date'] = data1['Datetime'].dt.date
data1

,Datetime,Valeur,date
0,2014-01-06 02:00:00,6.51,2014-01-06
1,2014-01-06 02:15:00,6.51,2014-01-06
2,2014-01-06 02:30:00,6.72,2014-01-06
3,2014-01-06 02:45:00,6.51,2014-01-06
4,2014-01-06 03:00:00,6.93,2014-01-06
...,...,...,...
209821,2020-06-30 23:00:00,5.04,2020-06-30
209822,2020-06-30 23:15:00,3.36,2020-06-30
209823,2020-06-30 23:30:00,3.36,2020-06-30
209824,2020-06-30 23:45:00,4.20,2020-06-30


In [71]:
dataplus = data1.merge(df_holidays, on='date', how='left')
dataplus['date'] = pd.to_datetime(dataplus['date'])
dataplus = dataplus.drop('Datetime_y', axis=1)
dataplus.columns = ['Datetime', 'Valeur', 'date', 'holiday']
#dataplus.loc[dataplus['date'] >= '2014-06-09']

In [72]:
dataplus['holiday'] = dataplus['holiday'].fillna(value=False)
#dataplus.loc[dataplus['Datetime'] >= '2014-06-09']

In [73]:
dataplus['working day'] =  dataplus['Datetime'].apply(cal.is_working_day)
dataplus.tail()

,Datetime,Valeur,date,holiday,working day
209821,2020-06-30 23:00:00,5.04,2020-06-30,False,True
209822,2020-06-30 23:15:00,3.36,2020-06-30,False,True
209823,2020-06-30 23:30:00,3.36,2020-06-30,False,True
209824,2020-06-30 23:45:00,4.20,2020-06-30,False,True
209825,2020-07-01 00:00:00,3.78,2020-07-01,False,True


In [79]:
dataplus = dataplus.drop('holiday', axis=1)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 209826 entries, 0 to 209825
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Datetime     209826 non-null  datetime64[ns]
 1   Valeur       209826 non-null  float64       
 2   date         209826 non-null  datetime64[ns]
 3   working day  209826 non-null  bool          
dtypes: bool(1), datetime64[ns](2), float64(1)
memory usage: 6.6 MB


In [86]:
#dataplus_save = dataplus.drop('date', axis=1)
dataplus_save.to_pickle('./building1_holidays.pkl')
dataplus_save.tail()

,Datetime,Valeur,working day
209821,2020-06-30 23:00:00,5.04,True
209822,2020-06-30 23:15:00,3.36,True
209823,2020-06-30 23:30:00,3.36,True
209824,2020-06-30 23:45:00,4.20,True
209825,2020-07-01 00:00:00,3.78,True


In [85]:
dataplus_save.info()
dataplus_save2 = dataplus_save.drop_duplicates(subset='Datetime')
dataplus_save2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209826 entries, 0 to 209825
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Datetime     209826 non-null  datetime64[ns]
 1   Valeur       209826 non-null  float64       
 2   working day  209826 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(1)
memory usage: 5.0 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 209826 entries, 0 to 209825
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Datetime     209826 non-null  datetime64[ns]
 1   Valeur       209826 non-null  float64       
 2   working day  209826 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(1)
memory usage: 5.0 MB


In [68]:
weather_rad = pd.read_csv('./weather_data/50.80_4.30_radiation.csv', parse_dates=[0])
weather_temp = pd.read_csv('./weather_data/50.80_4.30_temp.csv', parse_dates=[0])
weather_wind = pd.read_csv('./weather_data/50.80_4.30_wind_speed_10m.csv', parse_dates=[0])
weather_rad

,time,Radiation (Wh/m²)
0,2014-01-01 00:00:00,0.000000
1,2014-01-01 01:00:00,0.000069
2,2014-01-01 02:00:00,0.000000
3,2014-01-01 03:00:00,0.000000
4,2014-01-01 04:00:00,0.000000
...,...,...
56948,2020-06-30 20:00:00,8.437744
56949,2020-06-30 21:00:00,0.000000
56950,2020-06-30 22:00:00,0.000000
56951,2020-06-30 23:00:00,0.000000


In [32]:
weather = weather_rad.merge(weather_temp, on='time', how='left')
weather = weather.merge(weather_wind, on='time', how='left')

In [33]:
weather.head()

,time,Radiation (Wh/m²),Temperatures (°C),Wind speed (m/s)
0,2014-01-01 00:00:00,0.000000,6.534021,1.775049
1,2014-01-01 01:00:00,0.000069,6.450159,1.750478
2,2014-01-01 02:00:00,0.000000,6.074030,1.791114
3,2014-01-01 03:00:00,0.000000,5.751886,1.805243
4,2014-01-01 04:00:00,0.000000,5.327020,1.875873


In [34]:
weather['time'] = pd.to_datetime(weather['time'])
weather = weather.set_index(['time'])
weather.head()

,Radiation (Wh/m²),Temperatures (°C),Wind speed (m/s)
time,,,
2014-01-01 00:00:00,0.000000,6.534021,1.775049
2014-01-01 01:00:00,0.000069,6.450159,1.750478
2014-01-01 02:00:00,0.000000,6.074030,1.791114
2014-01-01 03:00:00,0.000000,5.751886,1.805243
2014-01-01 04:00:00,0.000000,5.327020,1.875873


In [35]:
weather15 = weather.resample('15T').pad()

In [36]:
weather15['Datetime'] = weather15.index
weather15.head()

,Radiation (Wh/m²),Temperatures (°C),Wind speed (m/s),Datetime
time,,,,
2014-01-01 00:00:00,0.000000,6.534021,1.775049,2014-01-01 00:00:00
2014-01-01 00:15:00,0.000000,6.534021,1.775049,2014-01-01 00:15:00
2014-01-01 00:30:00,0.000000,6.534021,1.775049,2014-01-01 00:30:00
2014-01-01 00:45:00,0.000000,6.534021,1.775049,2014-01-01 00:45:00
2014-01-01 01:00:00,0.000069,6.450159,1.750478,2014-01-01 01:00:00


In [41]:
data_B1 = dataplus.merge(weather15, on='Datetime')
data_B1.tail()

,Datetime,Valeur,date,working day,Radiation (Wh/m²),Temperatures (°C),Wind speed (m/s)
209821,2020-06-30 23:00:00,5.04,2020-06-30,True,0.000000,18.468164,1.596285
209822,2020-06-30 23:15:00,3.36,2020-06-30,True,0.000000,18.468164,1.596285
209823,2020-06-30 23:30:00,3.36,2020-06-30,True,0.000000,18.468164,1.596285
209824,2020-06-30 23:45:00,4.20,2020-06-30,True,0.000000,18.468164,1.596285
209825,2020-07-01 00:00:00,3.78,2020-07-01,True,0.026611,18.129449,1.559142


In [19]:
data_B1 = data_B1.drop('date', axis=1)

In [20]:
data_B1.to_csv('data_weather&holidays.csv')
data_B1.head()

,Datetime,Valeur,working day,Radiation (Wh/m²),Temperatures (°C),Wind speed (m/s)
0,2014-01-06 02:00:00,6.51,True,0.0,9.508508,3.071947
1,2014-01-06 02:15:00,6.51,True,0.0,9.508508,3.071947
2,2014-01-06 02:30:00,6.72,True,0.0,9.508508,3.071947
3,2014-01-06 02:45:00,6.51,True,0.0,9.508508,3.071947
4,2014-01-06 03:00:00,6.93,True,0.0,9.902185,2.997679


In [80]:
data_B1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209826 entries, 0 to 209825
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Datetime           209826 non-null  datetime64[ns]
 1   Valeur             209826 non-null  float64       
 2   date               209826 non-null  datetime64[ns]
 3   working day        209826 non-null  bool          
 4   Radiation (Wh/m²)  209826 non-null  float64       
 5   Temperatures (°C)  209826 non-null  float64       
 6   Wind speed (m/s)   209826 non-null  float64       
dtypes: bool(1), datetime64[ns](2), float64(4)
memory usage: 11.4 MB
